In [ ]:
import math
import os

import torch
import torch.nn.functional as F
import wandb
from gechebnet.data.dataloader import get_train_val_data_loaders
from gechebnet.engine.engine import create_supervised_evaluator, create_supervised_trainer
from gechebnet.engine.utils import prepare_batch, wandb_log
from gechebnet.graph.graph import HyperCubeGraph
from gechebnet.model.chebnet import ChebNet
from gechebnet.model.optimizer import get_optimizer
from gechebnet.utils import random_choice
from ignite.contrib.handlers import ProgressBar
from ignite.engine import Events
from ignite.metrics import Accuracy, Loss

DATA_PATH = "data"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DATASET = "MNIST"
VAL_RATIO = 0.2
NX1, NX2 = (28, 28)

IN_CHANNELS = 1
OUT_CHANNELS = 10
HIDDEN_CHANNELS = 20

EPOCHS = 20


def get_model(nx3, knn, eps, xi, weight_sigma, weight_kernel, K, pooling):
    graphs = [
        HyperCubeGraph(
            grid_size=(NX1, NX2),
            nx3=nx3,
            weight_kernel=weight_kernel,
            weight_sigma=weight_sigma,
            knn=knn,
            sigmas=(xi / eps, xi, 1.0),
            weight_comp_device=DEVICE,
        ),
        HyperCubeGraph(
            grid_size=(NX1 // 2, NX2 // 2),
            nx3=nx3,
            weight_kernel=weight_kernel,
            weight_sigma=weight_sigma,
            knn=knn,
            sigmas=(xi / eps, xi, 1.0),
            weight_comp_device=DEVICE,
        ),
        HyperCubeGraph(
            grid_size=(NX1 // 2 // 2, NX2 // 2 // 2),
            nx3=nx3,
            weight_kernel=weight_kernel,
            weight_sigma=weight_sigma,
            knn=knn,
            sigmas=(xi / eps, xi, 1.0),
            weight_comp_device=DEVICE,
        ),
    ]

    model = ChebNet(graphs, K, IN_CHANNELS, OUT_CHANNELS, HIDDEN_CHANNELS, laplacian_device=DEVICE, pooling=pooling)
    #while model.capacity < NUM_PARAMS:
    #    hidden_channels += 1
    #    model = ChebNet(graphs, K, IN_CHANNELS, OUT_CHANNELS, hidden_channels, laplacian_device=DEVICE, pooling=pooling)

    print(model.capacity)

    return model.to(DEVICE)


In [ ]:
batch_size = 16
xi = 0.01
eps = 0.1
K = 20
knn = 8
learning_rate = 1e-3
nx3 = 1
optimizer = "adam"
weight_sigma = 1.
weight_decay = 1e-6
weight_kernel = "gaussian"
pooling = "max"
    
train_loader, val_loader = get_train_val_data_loaders(DATASET, batch_size=batch_size, val_ratio=VAL_RATIO, data_path=DATA_PATH)

model = get_model(nx3, knn, eps, xi, weight_sigma, weight_kernel, K, pooling)

optimizer = get_optimizer(model, optimizer, learning_rate, weight_decay)

loss_fn = F.nll_loss
metrics = {"val_mnist_acc": Accuracy(), "val_mnist_loss": Loss(loss_fn)}

# create ignite's engines
trainer = create_supervised_trainer(
    nx=(NX1, NX2, nx3), model=model, optimizer=optimizer, loss_fn=F.nll_loss, device=DEVICE, prepare_batch=prepare_batch
)

evaluator = create_supervised_evaluator(nx=(NX1, NX2, nx3), model=model, metrics=metrics, device=DEVICE, prepare_batch=prepare_batch)

# track training with wandb
_ = trainer.add_event_handler(Events.EPOCH_COMPLETED, wandb_log, evaluator, val_loader)

# save best model
#trainer.run(train_loader, max_epochs=EPOCHS)

In [ ]:
from gechebnet.model.convolution import cheb_conv, ChebConv
from gechebnet.model.chebnet import ChebNet

In [ ]:
model = ChebNet(
[HyperCubeGraph(
            grid_size=(2, 2),
            nx3=2,
            weight_kernel="gaussian",
            weight_sigma=1.,
            knn=2,
            sigmas=(xi / eps, xi, 1.0),
            weight_comp_device=DEVICE,
        )],
    10, 1, 10, 20
)

model.capacity

In [ ]:
xi = 0.01
eps = 0.1

graph = HyperCubeGraph(
            grid_size=(2, 2),
            nx3=2,
            weight_kernel="gaussian",
            weight_sigma=1.,
            knn=2,
            sigmas=(xi / eps, xi, 1.0),
            weight_comp_device=DEVICE,
        )

In [ ]:
graph.laplacian

In [ ]:
x = torch.ones(1, 2, 8)
x

In [ ]:
w = torch.rand(2, 2, 3)
w

In [ ]:
cheb_conv(graph.laplacian, x, w).permute(1, 2, 0)

In [ ]:
import math

In [ ]:
math.exp(-1/(2*(0.1)**2))

In [ ]:
C = ChebConv(graph, 2, 3, 1)

In [ ]:
C.weight

In [ ]:
C.bias

In [ ]:
C.laplacian

In [ ]:
x = torch.ones(1, 2, 8)
C(x)

In [ ]:
x = torch.rand(1, 1, 2, 2)
x


In [ ]:

B, C, H, W = x.shape  # (B, C, H, W)

x = x.unsqueeze(2).expand(B, C, 2, 2, 2).reshape(B, C, -1)  # (B, C, L*H*W)
x

In [ ]:
x.reshape(B, C, -1, H, W)

In [ ]:
def mod_operator(x, mod, offset):
    return x - mod * ((x-offset)//mod)

In [ ]:
import numpy as np
for x in np.linspace(-math.pi, math.pi, 100):
    if mod_operator(x, math.pi, -math.pi/2) != modulo_operator(x, math.pi, -math.pi/2):
        print(x, mod_operator(x, math.pi, -math.pi/2), modulo_operator(x, math.pi, -math.pi/2) )

In [ ]:
def modulo_operator(x, mod, offset):
    if offset <= x < offset + mod:
        return x + 0.5*mod + offset
    
    if offset - mod <= x < offset:
        return x + 1.5*mod + offset
    
    if offset + mod <= x < offset + 2*mod:
        return x - 0.5*mod + offset
    
    return False

In [ ]:
modulo_operator(0, math.pi, -math.pi/2)

In [ ]:
def mod_operator(x, mod, offset):
    
    range1 = LazyTensor.step(-(x).abs() + math.pi / 2)  # - pi/2 <= y3 <= pi/2
    range2 = LazyTensor.step(xi[2] - xj[2] - math.pi / 2 - eps)  # y3 + eps <= x3 - pi/2
    range3 = LazyTensor.step(xj[2] - xi[2] - math.pi / 2 - eps)  # y3 - eps >= x3 + pi/2
    
    dx3 = range1 * (x) + range2 * (xj[2] - xi[2] + math.pi) + range3 * (xj[2] - xi[2] - math.pi)
